In [1]:
import time
import os

import scanpy
import numpy as np
import scipy.sparse as sp

import pickle
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import gc
from skimage import io
import umap
from sklearn.cluster import MiniBatchKMeans,AgglomerativeClustering
from sklearn.decomposition import PCA
from util.plot import plotembeddingbyCT,plotembeddingbyCT_contrast
import cv2
import sklearn.metrics 

from util.plot_new import *

In [2]:
radius=48
datadir='/media/xinyi/dcis2idc/data'
with open(os.path.join(datadir,'processed','train_cnnvae_names_valcores'), 'rb') as output:
    allImgNames=pickle.load(output)
with open(os.path.join(datadir,'processed','train_cnnvae_names_valsamples'), 'rb') as output:
    allImgNames_new=pickle.load(output)
name='exp0'
plotsavepath='/media/xinyi/dcis2idc/plots/cnnvae'+name
modelsavepath='/media/xinyi/dcis2idc/models/cnnvae'+name
plottype='umap'
sampledir=plotsavepath
clustersavedir_valcores=os.path.join(sampledir,'cluster_valcores_reordered')
clusterplotdir_valcores=os.path.join(clustersavedir_valcores,'plots')
clustersavedir_valsamples=os.path.join(sampledir,'cluster_valsamples_reordered')
clusterplotdir_valsamples=os.path.join(clustersavedir_valsamples,'plots')

with open(os.path.join(datadir,'processed','train_cnnvae_coord_valcores'), 'rb') as output:
    coordlist=pickle.load(output)
with open(os.path.join(datadir,'processed','train_cnnvae_coord_valsamples'), 'rb') as output:
    coordlist_new=pickle.load(output)
    
with open(os.path.join(datadir,'processed','train_cnnvae_cellLabels_valcores'), 'rb') as output:
    cellIDlist=pickle.load(output)
with open(os.path.join(datadir,'processed','train_cnnvae_cellLabels_valsamples'), 'rb') as output:
    cellIDlist_new=pickle.load(output)
    
neworder=[1, 5, 3, 7, 2, 0, 4, 6]  
subclusternumbers=[4,6,8,6,6,6,6,4]

ep=311
plotepoch=ep
ncluster=8
n_neighbors=10
min_dist=0.25
n_pcs=50
savenamecluster='minibatchkmean_ncluster'+str(ncluster)+'n_pcs'+str(n_pcs)+'epoch'+str(plotepoch)
with open(os.path.join(clustersavedir_valcores,savenamecluster+'_all'), 'rb') as output:
    clusterRes_reordered=pickle.load(output)
with open(os.path.join(clustersavedir_valsamples,'minibatchkmean_ncluster'+str(ncluster)+'n_pcs'+str(n_pcs)+'epoch'+str(plotepoch)+'_all'), 'rb') as output:
    clusterRes_reordered_new=pickle.load(output)

In [7]:
#plot by disease progression
br1003aSpecs=pd.read_excel('/media/xinyi/dcis2idc/data/BR1003a specs.xlsx',header=10)
br301Specs=pd.read_excel('/media/xinyi/dcis2idc/data/BR301 specs.xlsx',header=10)
br8018aSpecs=pd.read_excel('/media/xinyi/dcis2idc/data/BR8018a specs.xlsx',header=10)
br1003aSpecs.index=br1003aSpecs.loc[:,'Position']
br301Specs.index=br301Specs.loc[:,'Position']
br8018aSpecs.index=br8018aSpecs.loc[:,'Position']

progList=np.copy(allImgNames)
for s in np.unique(allImgNames):
    ssplit=s.split('_')
    if 'br1003a'==ssplit[0]:
        prog_s=br1003aSpecs.loc[(ssplit[-1],'Pathology diagnosis')]
    elif 'br301'==ssplit[0]:
        prog_s=br301Specs.loc[(ssplit[-1],'Pathology diagnosis')]
    elif 'br8018a'==ssplit[0]:
        prog_s=br8018aSpecs.loc[(ssplit[-1],'Pathology diagnosis')]
    progList[allImgNames==s]=prog_s
    
progList_new=np.copy(allImgNames_new)
for s in np.unique(allImgNames_new):
    ssplit=s.split('_')
    if 'br1003a'==ssplit[0]:
        prog_s=br1003aSpecs.loc[(ssplit[-1],'Pathology diagnosis')]
    elif 'br301'==ssplit[0]:
        prog_s=br301Specs.loc[(ssplit[-1],'Pathology diagnosis')]
    elif 'br8018a'==ssplit[0]:
        prog_s=br8018aSpecs.loc[(ssplit[-1],'Pathology diagnosis')]
    progList_new[allImgNames_new==s]=prog_s
    


## plot umap

In [4]:
ncluster=8
n_neighbors=10
min_dist=0.25
n_pcs=50


savenameAdd='_nn'+str(n_neighbors)+'mdist0'+str(int(min_dist*100))+'epoch'+str(ep)
with open(os.path.join(modelsavepath,'umapEmbedding_plottingIdx_progBalanced_'+str(0)+savenameAdd+'_valCores'), 'rb') as output:
    embedding=pickle.load(output)
with open(os.path.join(modelsavepath,'umapEmbedding_plottingIdx_progBalanced_'+str(0)+savenameAdd+'_valSamples'), 'rb') as output:
    embedding_new=pickle.load(output)
    
plotsavenameAdd='_all'
plotembeddingbyCT(clusterRes_reordered,'minibatchkmean',[],embedding,clusterplotdir_valcores,plottype+' of all samples',savenameAdd=savenamecluster+plotsavenameAdd,colorseq=neworder)
plotembeddingbyCT_contrast(clusterRes_reordered,'minibatchkmean',[],embedding,os.path.join(clusterplotdir_valcores,'contrast'),plottype+' of all samples',savenameAdd=savenamecluster+plotsavenameAdd,maxplot=50)

plotembeddingbyCT_str(progList,'prog',[],embedding,clusterplotdir_valcores,plottype+' of all samples',savenameAdd=savenamecluster+plotsavenameAdd)
# plotembeddingbyCT_contrast(progList,'prog',[],embedding,os.path.join(clusterplotdir_valcores,'contrast'),plottype+' of all samples',savenameAdd=savenamecluster+plotsavenameAdd,maxplot=50)

plotembeddingbyCT(clusterRes_reordered_new,'minibatchkmean',[],embedding_new,clusterplotdir_valsamples,plottype+' of all samples',savenameAdd=savenamecluster+plotsavenameAdd,colorseq=neworder)
plotembeddingbyCT_contrast(clusterRes_reordered_new,'minibatchkmean',[],embedding_new,os.path.join(clusterplotdir_valsamples,'contrast'),plottype+' of all samples',savenameAdd=savenamecluster+plotsavenameAdd,maxplot=50)

plotembeddingbyCT_str(progList_new,'prog',[],embedding_new,clusterplotdir_valsamples,plottype+' of all samples',savenameAdd=savenamecluster+plotsavenameAdd)
# plotembeddingbyCT_contrast(progList_new,'prog',[],embedding_new,os.path.join(clusterplotdir_valsamples,'contrast'),plottype+' of all samples',savenameAdd=savenamecluster+plotsavenameAdd,maxplot=50)




In [12]:
subclusterplotdir_valcores

'/media/xinyi/dcis2idc/plots/cnnvaeexp0/cluster_valcores_reordered/minibatchkmean_ncluster8n_pcs50epoch311_plottingIdx_progBalanced_0_subcluster6/plots'

In [13]:
for cnew in np.unique(clusterRes_reordered):
    print(cnew)
    c=neworder[cnew]
    subclustersavedir_valcores=os.path.join(clustersavedir_valcores,savenamecluster+'_plottingIdx_progBalanced_'+str(0)+'_subcluster'+str(c))
    subclusterplotdir_valcores=os.path.join(subclustersavedir_valcores,'plots')
    with open(os.path.join(subclustersavedir_valcores,'minibatchkmean_ncluster'+str(subclusternumbers[c])+'n_pcs'+str(n_pcs)+'epoch'+str(plotepoch)+'_all'), 'rb') as output:
        clusterRes_sub=pickle.load(output)

    plotembeddingbyCT(clusterRes_sub,'minibatchkmean',[],embedding[clusterRes_reordered==cnew],subclusterplotdir_valcores,plottype+' of all samples',savenameAdd=savenamecluster)
    plotembeddingbyCT_contrast(clusterRes_sub,'minibatchkmean',[],embedding[clusterRes_reordered==cnew],os.path.join(subclusterplotdir_valcores,'contrast'),plottype+' of all samples',savenameAdd=savenamecluster,maxplot=50)

    plotembeddingbyCT_str(progList[clusterRes_reordered==cnew],'prog',[],embedding[clusterRes_reordered==cnew],subclusterplotdir_valcores,plottype+' of all samples',savenameAdd=savenamecluster)
#     plotembeddingbyCT_contrast(progList[clusterRes_reordered==cnew],'prog',[],embedding[clusterRes_reordered==cnew],os.path.join(subclusterplotdir_valcores,'contrast'),plottype+' of all samples',savenameAdd=savenamecluster,maxplot=50)

for cnew in np.unique(clusterRes_reordered_new):
    c=neworder[cnew]
    subclustersavedir_valsamples=os.path.join(clustersavedir_valsamples,savenamecluster+'_plottingIdx_progBalanced_'+str(0)+'_subcluster'+str(c))
    subclusterplotdir_valsamples=os.path.join(subclustersavedir_valsamples,'plots')
    with open(os.path.join(subclustersavedir_valsamples,'minibatchkmean_ncluster'+str(subclusternumbers[c])+'n_pcs'+str(n_pcs)+'epoch'+str(plotepoch)+'_all'), 'rb') as output:
        clusterRes_sub=pickle.load(output)
    
    plotembeddingbyCT(clusterRes_sub,'minibatchkmean',[],embedding_new[clusterRes_reordered_new==cnew],subclusterplotdir_valsamples,plottype+' of all samples',savenameAdd=savenamecluster)
    plotembeddingbyCT_contrast(clusterRes_sub,'minibatchkmean',[],embedding_new[clusterRes_reordered_new==cnew],os.path.join(subclusterplotdir_valsamples,'contrast'),plottype+' of all samples',savenameAdd=savenamecluster,maxplot=50)

    plotembeddingbyCT_str(progList_new[clusterRes_reordered_new==cnew],'prog',[],embedding_new[clusterRes_reordered_new==cnew],subclusterplotdir_valsamples,plottype+' of all samples',savenameAdd=savenamecluster)
#     plotembeddingbyCT_contrast(progList_new[clusterRes_reordered_new==cnew],'prog',[],embedding_new[clusterRes_reordered_new==cnew],os.path.join(subclusterplotdir_valsamples,'contrast'),plottype+' of all samples',savenameAdd=savenamecluster,maxplot=50)


0
1
2
3
4
5
6
7


## cluster vs pathology

In [6]:
progNames,progCounts=np.unique(progList[progList!='Hyperplasia with saccular dilatation'],return_counts=True)
progSampleRate={}
for p in range(progNames.size):
    progSampleRate[progNames[p]]=max(0.1,np.min(progCounts)/progCounts[p])
    
plotepoch=311
np.random.seed(6)
plottingIdx_i=np.array([])
n_pcs=50
uniqueImgNames,imgNameIdx=np.unique(allImgNames,return_index=True)
for sidx in range(uniqueImgNames.size):
    s=uniqueImgNames[sidx]
    p=progList[imgNameIdx[sidx]]
    print(s+' '+p)
    if p=='Hyperplasia with saccular dilatation':
        rate=1
    else:
        rate=progSampleRate[p]
    nsamples=int(np.sum(allImgNames==s)*rate)
    plottingIdx_i=np.concatenate((plottingIdx_i,
                                np.random.choice(np.arange(allImgNames.shape[0])[allImgNames==s],nsamples,replace=False)))



plotsavenameAdd_i='_plottingIdx_progBalanced_'+str(0)
plottingIdx_i=plottingIdx_i.astype(int)

br1003a_1_cytokeratin_555_aSMA_647_hoechst_B1 Hyperplasia
br1003a_1_cytokeratin_555_aSMA_647_hoechst_B10 Hyperplasia
br1003a_1_cytokeratin_555_aSMA_647_hoechst_B2 Hyperplasia
br1003a_1_cytokeratin_555_aSMA_647_hoechst_B3 Hyperplasia
br1003a_1_cytokeratin_555_aSMA_647_hoechst_B4 Hyperplasia with saccular dilatation
br1003a_1_cytokeratin_555_aSMA_647_hoechst_B5 Hyperplasia
br1003a_1_cytokeratin_555_aSMA_647_hoechst_B7 Hyperplasia
br1003a_1_cytokeratin_555_aSMA_647_hoechst_B8 Hyperplasia
br1003a_1_cytokeratin_555_aSMA_647_hoechst_B9 Hyperplasia
br1003a_1_cytokeratin_555_aSMA_647_hoechst_D1 Hyperplasia
br1003a_1_cytokeratin_555_aSMA_647_hoechst_D2 Hyperplasia
br1003a_1_cytokeratin_555_aSMA_647_hoechst_D3 Atypical hyperplasia
br1003a_1_cytokeratin_555_aSMA_647_hoechst_D4 Atypical hyperplasia
br1003a_1_cytokeratin_555_aSMA_647_hoechst_D7 Atypical hyperplasia
br1003a_1_cytokeratin_555_aSMA_647_hoechst_J1 Breast tissue
br1003a_1_cytokeratin_555_aSMA_647_hoechst_J10 Breast tissue
br1003a_1_cyto

br8018a_3_collagen1_647_hoechst_E3 Invasive ductal carcinoma
br8018a_3_collagen1_647_hoechst_E5 Invasive ductal carcinoma
br8018a_3_collagen1_647_hoechst_E8 Invasive ductal carcinoma
br8018a_3_collagen1_647_hoechst_E9 Invasive ductal carcinoma
br8018a_3_collagen1_647_hoechst_G1 Invasive ductal carcinoma
br8018a_3_collagen1_647_hoechst_G10 Invasive ductal carcinoma
br8018a_3_collagen1_647_hoechst_G2 Invasive ductal carcinoma
br8018a_3_collagen1_647_hoechst_G3 Invasive ductal carcinoma
br8018a_3_collagen1_647_hoechst_G4 Invasive ductal carcinoma
br8018a_3_collagen1_647_hoechst_G5 Invasive ductal carcinoma
br8018a_3_collagen1_647_hoechst_G6 Invasive ductal carcinoma
br8018a_3_collagen1_647_hoechst_G8 Invasive ductal carcinoma
br8018a_4_cytokeratin_555_gh2ax_647_hoechst_C1 Invasive ductal carcinoma
br8018a_4_cytokeratin_555_gh2ax_647_hoechst_C2 Invasive ductal carcinoma
br8018a_4_cytokeratin_555_gh2ax_647_hoechst_C4 Invasive ductal carcinoma
br8018a_4_cytokeratin_555_gh2ax_647_hoechst_C5 I

In [5]:
progNames,progCounts=np.unique(progList,return_counts=True)
progSampleRate={}
for p in range(progNames.size):
    progSampleRate[progNames[p]]=np.min(progCounts)/progCounts[p]
    
plotepoch=311
np.random.seed(6)
plottingIdx_prop=np.array([])
n_pcs=50
uniqueImgNames,imgNameIdx=np.unique(allImgNames,return_index=True)
for sidx in range(uniqueImgNames.size):
    s=uniqueImgNames[sidx]
    p=progList[imgNameIdx[sidx]]
    print(s+' '+p)
    nsamples=int(np.sum(allImgNames==s)*progSampleRate[p])
    plottingIdx_prop=np.concatenate((plottingIdx_prop,
                                np.random.choice(np.arange(allImgNames.shape[0])[allImgNames==s],nsamples,replace=False)))



plotsavenameAdd_prop='_plottingIdx_progBalanced_prop'
plottingIdx_prop=plottingIdx_prop.astype(int)

br1003a_1_cytokeratin_555_aSMA_647_hoechst_B1 Hyperplasia
br1003a_1_cytokeratin_555_aSMA_647_hoechst_B10 Hyperplasia
br1003a_1_cytokeratin_555_aSMA_647_hoechst_B2 Hyperplasia
br1003a_1_cytokeratin_555_aSMA_647_hoechst_B3 Hyperplasia
br1003a_1_cytokeratin_555_aSMA_647_hoechst_B4 Hyperplasia with saccular dilatation
br1003a_1_cytokeratin_555_aSMA_647_hoechst_B5 Hyperplasia
br1003a_1_cytokeratin_555_aSMA_647_hoechst_B7 Hyperplasia
br1003a_1_cytokeratin_555_aSMA_647_hoechst_B8 Hyperplasia
br1003a_1_cytokeratin_555_aSMA_647_hoechst_B9 Hyperplasia
br1003a_1_cytokeratin_555_aSMA_647_hoechst_D1 Hyperplasia
br1003a_1_cytokeratin_555_aSMA_647_hoechst_D2 Hyperplasia
br1003a_1_cytokeratin_555_aSMA_647_hoechst_D3 Atypical hyperplasia
br1003a_1_cytokeratin_555_aSMA_647_hoechst_D4 Atypical hyperplasia
br1003a_1_cytokeratin_555_aSMA_647_hoechst_D7 Atypical hyperplasia
br1003a_1_cytokeratin_555_aSMA_647_hoechst_J1 Breast tissue
br1003a_1_cytokeratin_555_aSMA_647_hoechst_J10 Breast tissue
br1003a_1_cyto

br8018a_3_collagen1_647_hoechst_D9 Invasive ductal carcinoma
br8018a_3_collagen1_647_hoechst_E1 Invasive ductal carcinoma
br8018a_3_collagen1_647_hoechst_E10 Invasive ductal carcinoma
br8018a_3_collagen1_647_hoechst_E2 Invasive ductal carcinoma
br8018a_3_collagen1_647_hoechst_E3 Invasive ductal carcinoma
br8018a_3_collagen1_647_hoechst_E5 Invasive ductal carcinoma
br8018a_3_collagen1_647_hoechst_E8 Invasive ductal carcinoma
br8018a_3_collagen1_647_hoechst_E9 Invasive ductal carcinoma
br8018a_3_collagen1_647_hoechst_G1 Invasive ductal carcinoma
br8018a_3_collagen1_647_hoechst_G10 Invasive ductal carcinoma
br8018a_3_collagen1_647_hoechst_G2 Invasive ductal carcinoma
br8018a_3_collagen1_647_hoechst_G3 Invasive ductal carcinoma
br8018a_3_collagen1_647_hoechst_G4 Invasive ductal carcinoma
br8018a_3_collagen1_647_hoechst_G5 Invasive ductal carcinoma
br8018a_3_collagen1_647_hoechst_G6 Invasive ductal carcinoma
br8018a_3_collagen1_647_hoechst_G8 Invasive ductal carcinoma
br8018a_4_cytokeratin_

In [8]:
ctorder=np.array(['Breast tissue','Cancer adjacent normal breast tissue', 'Hyperplasia','Hyperplasia with saccular dilatation','Atypical hyperplasia','Invasive ductal carcinoma (breast tissue)','Invasive ductal carcinoma'])
plotCTcomp(clusterRes_reordered[plottingIdx_prop],progList[plottingIdx_prop],clusterplotdir_valcores,'pathologyHeatmap',False,ctorder=ctorder)
plotCTcomp(clusterRes_reordered[plottingIdx_prop],progList[plottingIdx_prop],clusterplotdir_valcores,'pathologyHeatmap',True,ctorder=ctorder)

for cnew in np.unique(clusterRes_reordered):
    subclusterRes=np.zeros_like(clusterRes_reordered)-1
    c=neworder[cnew]
    subclustersavedir_valcores=os.path.join(clustersavedir_valcores,savenamecluster+plotsavenameAdd_i+'_subcluster'+str(c))
    subclusterplotdir_valcores=os.path.join(subclustersavedir_valcores,'plots')
    with open(os.path.join(subclustersavedir_valcores,'minibatchkmean_ncluster'+str(subclusternumbers[c])+'n_pcs'+str(n_pcs)+'epoch'+str(plotepoch)+'_all'), 'rb') as output:
        subclusterRes[clusterRes_reordered==cnew]=pickle.load(output)
    plotCTcomp(subclusterRes[plottingIdx_prop][clusterRes_reordered[plottingIdx_prop]==cnew],progList[plottingIdx_prop][clusterRes_reordered[plottingIdx_prop]==cnew],subclusterplotdir_valcores,'pathologyHeatmap_ncluster'+str(subclusternumbers[c]),False,ctorder=ctorder)
    plotCTcomp(subclusterRes[plottingIdx_prop][clusterRes_reordered[plottingIdx_prop]==cnew],progList[plottingIdx_prop][clusterRes_reordered[plottingIdx_prop]==cnew],subclusterplotdir_valcores,'pathologyHeatmap_ncluster'+str(subclusternumbers[c]),True,ctorder=ctorder)
    gc.collect()


In [10]:
progNames,progCounts=np.unique(progList_new[progList_new!='Hyperplasia with saccular dilatation'],return_counts=True)
progSampleRate={}
for p in range(progNames.size):
    progSampleRate[progNames[p]]=max(0.1,np.min(progCounts)/progCounts[p])
    
plotepoch=311
np.random.seed(6)
plottingIdx_i_new=np.array([])
n_pcs=50
uniqueImgNames,imgNameIdx=np.unique(allImgNames_new,return_index=True)
for sidx in range(uniqueImgNames.size):
    s=uniqueImgNames[sidx]
    p=progList_new[imgNameIdx[sidx]]
    print(s+' '+p)
    if p=='Hyperplasia with saccular dilatation':
        rate=1
    else:
        rate=progSampleRate[p]
    nsamples=int(np.sum(allImgNames_new==s)*rate)
    plottingIdx_i_new=np.concatenate((plottingIdx_i_new,
                                np.random.choice(np.arange(allImgNames_new.shape[0])[allImgNames_new==s],nsamples,replace=False)))

plottingIdx_i_new=plottingIdx_i_new.astype(int)

br1003a_2_cytokeratin_555_ki67_647_hoechst_A1 Hyperplasia
br1003a_2_cytokeratin_555_ki67_647_hoechst_A10 Hyperplasia
br1003a_2_cytokeratin_555_ki67_647_hoechst_A2 Hyperplasia
br1003a_2_cytokeratin_555_ki67_647_hoechst_A3 Hyperplasia
br1003a_2_cytokeratin_555_ki67_647_hoechst_A5 Hyperplasia
br1003a_2_cytokeratin_555_ki67_647_hoechst_A7 Hyperplasia
br1003a_2_cytokeratin_555_ki67_647_hoechst_A8 Hyperplasia
br1003a_2_cytokeratin_555_ki67_647_hoechst_B1 Hyperplasia
br1003a_2_cytokeratin_555_ki67_647_hoechst_B10 Hyperplasia
br1003a_2_cytokeratin_555_ki67_647_hoechst_B2 Hyperplasia
br1003a_2_cytokeratin_555_ki67_647_hoechst_B3 Hyperplasia
br1003a_2_cytokeratin_555_ki67_647_hoechst_B4 Hyperplasia with saccular dilatation
br1003a_2_cytokeratin_555_ki67_647_hoechst_B5 Hyperplasia
br1003a_2_cytokeratin_555_ki67_647_hoechst_B6 Hyperplasia
br1003a_2_cytokeratin_555_ki67_647_hoechst_B7 Hyperplasia
br1003a_2_cytokeratin_555_ki67_647_hoechst_B8 Hyperplasia
br1003a_2_cytokeratin_555_ki67_647_hoechst_B9

br8018a_2_cytokeratin_555_ki67_647_hoechst_A3 Invasive ductal carcinoma
br8018a_2_cytokeratin_555_ki67_647_hoechst_A4 Invasive ductal carcinoma
br8018a_2_cytokeratin_555_ki67_647_hoechst_A5 Invasive ductal carcinoma
br8018a_2_cytokeratin_555_ki67_647_hoechst_A6 Invasive ductal carcinoma
br8018a_2_cytokeratin_555_ki67_647_hoechst_A7 Invasive ductal carcinoma
br8018a_2_cytokeratin_555_ki67_647_hoechst_A8 Invasive ductal carcinoma
br8018a_2_cytokeratin_555_ki67_647_hoechst_A9 Invasive ductal carcinoma
br8018a_2_cytokeratin_555_ki67_647_hoechst_B1 Invasive ductal carcinoma
br8018a_2_cytokeratin_555_ki67_647_hoechst_B10 Invasive ductal carcinoma
br8018a_2_cytokeratin_555_ki67_647_hoechst_B2 Invasive ductal carcinoma
br8018a_2_cytokeratin_555_ki67_647_hoechst_B3 Invasive ductal carcinoma
br8018a_2_cytokeratin_555_ki67_647_hoechst_B5 Invasive ductal carcinoma
br8018a_2_cytokeratin_555_ki67_647_hoechst_B6 Invasive ductal carcinoma (breast tissue)
br8018a_2_cytokeratin_555_ki67_647_hoechst_B7 I

In [9]:
progNames,progCounts=np.unique(progList_new,return_counts=True)
progSampleRate={}
for p in range(progNames.size):
    progSampleRate[progNames[p]]=np.min(progCounts)/progCounts[p]
    
plotepoch=311
np.random.seed(6)
plottingIdx_prop_new=np.array([])
n_pcs=50
uniqueImgNames,imgNameIdx=np.unique(allImgNames_new,return_index=True)
for sidx in range(uniqueImgNames.size):
    s=uniqueImgNames[sidx]
    p=progList_new[imgNameIdx[sidx]]
    print(s+' '+p)
    nsamples=int(np.sum(allImgNames_new==s)*progSampleRate[p])
    plottingIdx_prop_new=np.concatenate((plottingIdx_prop_new,
                                np.random.choice(np.arange(allImgNames_new.shape[0])[allImgNames_new==s],nsamples,replace=False)))

plottingIdx_prop_new=plottingIdx_prop_new.astype(int)

br1003a_2_cytokeratin_555_ki67_647_hoechst_A1 Hyperplasia
br1003a_2_cytokeratin_555_ki67_647_hoechst_A10 Hyperplasia
br1003a_2_cytokeratin_555_ki67_647_hoechst_A2 Hyperplasia
br1003a_2_cytokeratin_555_ki67_647_hoechst_A3 Hyperplasia
br1003a_2_cytokeratin_555_ki67_647_hoechst_A5 Hyperplasia
br1003a_2_cytokeratin_555_ki67_647_hoechst_A7 Hyperplasia
br1003a_2_cytokeratin_555_ki67_647_hoechst_A8 Hyperplasia
br1003a_2_cytokeratin_555_ki67_647_hoechst_B1 Hyperplasia
br1003a_2_cytokeratin_555_ki67_647_hoechst_B10 Hyperplasia
br1003a_2_cytokeratin_555_ki67_647_hoechst_B2 Hyperplasia
br1003a_2_cytokeratin_555_ki67_647_hoechst_B3 Hyperplasia
br1003a_2_cytokeratin_555_ki67_647_hoechst_B4 Hyperplasia with saccular dilatation
br1003a_2_cytokeratin_555_ki67_647_hoechst_B5 Hyperplasia
br1003a_2_cytokeratin_555_ki67_647_hoechst_B6 Hyperplasia
br1003a_2_cytokeratin_555_ki67_647_hoechst_B7 Hyperplasia
br1003a_2_cytokeratin_555_ki67_647_hoechst_B8 Hyperplasia
br1003a_2_cytokeratin_555_ki67_647_hoechst_B9

br8018a_2_cytokeratin_555_ki67_647_hoechst_A3 Invasive ductal carcinoma
br8018a_2_cytokeratin_555_ki67_647_hoechst_A4 Invasive ductal carcinoma
br8018a_2_cytokeratin_555_ki67_647_hoechst_A5 Invasive ductal carcinoma
br8018a_2_cytokeratin_555_ki67_647_hoechst_A6 Invasive ductal carcinoma
br8018a_2_cytokeratin_555_ki67_647_hoechst_A7 Invasive ductal carcinoma
br8018a_2_cytokeratin_555_ki67_647_hoechst_A8 Invasive ductal carcinoma
br8018a_2_cytokeratin_555_ki67_647_hoechst_A9 Invasive ductal carcinoma
br8018a_2_cytokeratin_555_ki67_647_hoechst_B1 Invasive ductal carcinoma
br8018a_2_cytokeratin_555_ki67_647_hoechst_B10 Invasive ductal carcinoma
br8018a_2_cytokeratin_555_ki67_647_hoechst_B2 Invasive ductal carcinoma
br8018a_2_cytokeratin_555_ki67_647_hoechst_B3 Invasive ductal carcinoma
br8018a_2_cytokeratin_555_ki67_647_hoechst_B5 Invasive ductal carcinoma
br8018a_2_cytokeratin_555_ki67_647_hoechst_B6 Invasive ductal carcinoma (breast tissue)
br8018a_2_cytokeratin_555_ki67_647_hoechst_B7 I

In [11]:
ctorder_new=np.array(['Breast tissue','Cancer adjacent normal breast tissue','Invasive ductal carcinoma (breast tissue)','Hyperplasia','Hyperplasia with saccular dilatation','Atypical hyperplasia','Ductal carcinoma in situ and breast tissue','Ductal carcinoma in situ','Ductal carcinoma in situ with early infiltrati', 'Micropapillary type ductal carcinoma in situ w','Invasive ductal carcinoma and breast tissue','Invasive ductal carcinoma'])
plotCTcomp(clusterRes_reordered_new[plottingIdx_prop_new],progList_new[plottingIdx_prop_new],clusterplotdir_valsamples,'pathologyHeatmap',False,ctorder=ctorder_new)
plotCTcomp(clusterRes_reordered_new[plottingIdx_prop_new],progList_new[plottingIdx_prop_new],clusterplotdir_valsamples,'pathologyHeatmap',True,ctorder=ctorder_new)

for cnew in np.unique(clusterRes_reordered_new):
    subclusterRes=np.zeros_like(clusterRes_reordered_new)-1
    c=neworder[cnew]
    subclustersavedir_valsamples=os.path.join(clustersavedir_valsamples,savenamecluster+plotsavenameAdd_i+'_subcluster'+str(c))
    subclusterplotdir_valsamples=os.path.join(subclustersavedir_valsamples,'plots')
    with open(os.path.join(subclustersavedir_valsamples,'minibatchkmean_ncluster'+str(subclusternumbers[c])+'n_pcs'+str(n_pcs)+'epoch'+str(plotepoch)+'_all'), 'rb') as output:
        subclusterRes[clusterRes_reordered_new==cnew]=pickle.load(output)
    plotCTcomp(subclusterRes[plottingIdx_prop_new][clusterRes_reordered_new[plottingIdx_prop_new]==cnew],progList_new[plottingIdx_prop_new][clusterRes_reordered_new[plottingIdx_prop_new]==cnew],subclusterplotdir_valsamples,'pathologyHeatmap_ncluster'+str(subclusternumbers[c]),False,ctorder=ctorder_new)
    plotCTcomp(subclusterRes[plottingIdx_prop_new][clusterRes_reordered_new[plottingIdx_prop_new]==cnew],progList_new[plottingIdx_prop_new][clusterRes_reordered_new[plottingIdx_prop_new]==cnew],subclusterplotdir_valsamples,'pathologyHeatmap_ncluster'+str(subclusternumbers[c]),True,ctorder=ctorder_new)
    gc.collect()



## cluster vs proteins

In [20]:
#all
stats2plot=['int_median','kurtosis','skewness','int_mode','entropy']
stains=['aSMA','cytokeratin','collagen1','gh2ax','ki67']
uniquenames,nameIdx=np.unique(allImgNames,return_index=True)

allstats=None
alllabels=None
allvarnames=None
for sidx in range(uniquenames.size):
    s=np.unique(allImgNames)[sidx]
    plottingIdx_i_s=np.arange(allImgNames.size)[allImgNames==s]-nameIdx[sidx]

    print(s)
    assert np.min(plottingIdx_i_s)>=0
#         ssplit=s.split('_')
    slabels=clusterRes_reordered[allImgNames==s]
    for stain_s in stains:
        if not stain_s in s:
            continue
        path_s=os.path.join(datadir,'_'.join(s.split('_')[:-1]),'cellular_'+stain_s.lower(),s.split('_')[-1] +'.csv')
        if not os.path.exists(path_s):
            continue
        print(stain_s)
        stats_s=pd.read_csv(path_s)
        stats_s.index=stats_s.loc[:,'label']
        if allstats is None:
            allstats=stats_s.loc[cellIDlist[s][plottingIdx_i_s],stats2plot].to_numpy()
            alllabels=np.copy(slabels)
            allvarnames=np.repeat(stain_s,slabels.size)
        else:
            allstats=np.concatenate((allstats,stats_s.loc[cellIDlist[s][plottingIdx_i_s],stats2plot].to_numpy()),axis=0)
            alllabels=np.concatenate((alllabels,np.copy(slabels)))
            allvarnames=np.concatenate((allvarnames,np.repeat(stain_s,slabels.size)))
plotCTcomp_exprs(stats2plot,alllabels,allstats,allvarnames,clusterplotdir_valcores,'proteinHeatmap_all',True)
plotCTcomp_exprs(stats2plot,alllabels,allstats,allvarnames,clusterplotdir_valcores,'proteinHeatmap_all',False)

            

br1003a_1_cytokeratin_555_aSMA_647_hoechst_B1
aSMA
cytokeratin
br1003a_1_cytokeratin_555_aSMA_647_hoechst_B10
aSMA
cytokeratin
br1003a_1_cytokeratin_555_aSMA_647_hoechst_B2
aSMA
cytokeratin
br1003a_1_cytokeratin_555_aSMA_647_hoechst_B3
aSMA
cytokeratin
br1003a_1_cytokeratin_555_aSMA_647_hoechst_B4
aSMA
cytokeratin
br1003a_1_cytokeratin_555_aSMA_647_hoechst_B5
aSMA
cytokeratin
br1003a_1_cytokeratin_555_aSMA_647_hoechst_B7
aSMA
cytokeratin
br1003a_1_cytokeratin_555_aSMA_647_hoechst_B8
aSMA
cytokeratin
br1003a_1_cytokeratin_555_aSMA_647_hoechst_B9
aSMA
cytokeratin
br1003a_1_cytokeratin_555_aSMA_647_hoechst_D1
aSMA
cytokeratin
br1003a_1_cytokeratin_555_aSMA_647_hoechst_D2
aSMA
cytokeratin
br1003a_1_cytokeratin_555_aSMA_647_hoechst_D3
aSMA
cytokeratin
br1003a_1_cytokeratin_555_aSMA_647_hoechst_D4
aSMA
cytokeratin
br1003a_1_cytokeratin_555_aSMA_647_hoechst_D7
aSMA
cytokeratin
br1003a_1_cytokeratin_555_aSMA_647_hoechst_J1
aSMA
cytokeratin
br1003a_1_cytokeratin_555_aSMA_647_hoechst_J10
aSMA
cy

gh2ax
br8018a_4_cytokeratin_555_gh2ax_647_hoechst_C2
cytokeratin
gh2ax
br8018a_4_cytokeratin_555_gh2ax_647_hoechst_C4
cytokeratin
gh2ax
br8018a_4_cytokeratin_555_gh2ax_647_hoechst_C5
cytokeratin
gh2ax
br8018a_4_cytokeratin_555_gh2ax_647_hoechst_C6
cytokeratin
gh2ax
br8018a_4_cytokeratin_555_gh2ax_647_hoechst_C7
cytokeratin
gh2ax
br8018a_4_cytokeratin_555_gh2ax_647_hoechst_C8
cytokeratin
gh2ax
br8018a_4_cytokeratin_555_gh2ax_647_hoechst_C9
cytokeratin
gh2ax
br8018a_4_cytokeratin_555_gh2ax_647_hoechst_D1
cytokeratin
gh2ax
br8018a_4_cytokeratin_555_gh2ax_647_hoechst_D3
cytokeratin
gh2ax
br8018a_4_cytokeratin_555_gh2ax_647_hoechst_D5
cytokeratin
gh2ax
br8018a_4_cytokeratin_555_gh2ax_647_hoechst_D8
cytokeratin
gh2ax
br8018a_4_cytokeratin_555_gh2ax_647_hoechst_D9
cytokeratin
gh2ax
br8018a_4_cytokeratin_555_gh2ax_647_hoechst_E1
cytokeratin
gh2ax
br8018a_4_cytokeratin_555_gh2ax_647_hoechst_E3
cytokeratin
gh2ax
br8018a_4_cytokeratin_555_gh2ax_647_hoechst_E6
cytokeratin
gh2ax
br8018a_4_cytokerat

In [18]:
#equal proportion of pathologies
stats2plot=['int_median','kurtosis','skewness','int_mode','entropy']
stains=['aSMA','cytokeratin','collagen1','gh2ax','ki67']
uniquenames,nameIdx=np.unique(allImgNames,return_index=True)

allstats=None
alllabels=None
allvarnames=None
for sidx in range(uniquenames.size):
    s=np.unique(allImgNames)[sidx]
    plottingIdx_i_s=plottingIdx_prop[allImgNames[plottingIdx_prop]==s]-nameIdx[sidx]

    print(s)
    assert np.min(plottingIdx_i_s)>=0
#         ssplit=s.split('_')
    slabels=clusterRes_reordered[plottingIdx_prop][allImgNames[plottingIdx_prop]==s]
    for stain_s in stains:
        if not stain_s in s:
            continue
        path_s=os.path.join(datadir,'_'.join(s.split('_')[:-1]),'cellular_'+stain_s.lower(),s.split('_')[-1] +'.csv')
        if not os.path.exists(path_s):
            continue
        print(stain_s)
        stats_s=pd.read_csv(path_s)
        stats_s.index=stats_s.loc[:,'label']
        if allstats is None:
            allstats=stats_s.loc[cellIDlist[s][plottingIdx_i_s],stats2plot].to_numpy()
            alllabels=np.copy(slabels)
            allvarnames=np.repeat(stain_s,slabels.size)
        else:
            allstats=np.concatenate((allstats,stats_s.loc[cellIDlist[s][plottingIdx_i_s],stats2plot].to_numpy()),axis=0)
            alllabels=np.concatenate((alllabels,np.copy(slabels)))
            allvarnames=np.concatenate((allvarnames,np.repeat(stain_s,slabels.size)))
plotCTcomp_exprs(stats2plot,alllabels,allstats,allvarnames,clusterplotdir_valcores,'proteinHeatmap_prop',True)
plotCTcomp_exprs(stats2plot,alllabels,allstats,allvarnames,clusterplotdir_valcores,'proteinHeatmap_prop',False)

            

br1003a_1_cytokeratin_555_aSMA_647_hoechst_B1
aSMA
cytokeratin
br1003a_1_cytokeratin_555_aSMA_647_hoechst_B10
aSMA
cytokeratin
br1003a_1_cytokeratin_555_aSMA_647_hoechst_B2
aSMA
cytokeratin
br1003a_1_cytokeratin_555_aSMA_647_hoechst_B3
aSMA
cytokeratin
br1003a_1_cytokeratin_555_aSMA_647_hoechst_B4
aSMA
cytokeratin
br1003a_1_cytokeratin_555_aSMA_647_hoechst_B5
aSMA
cytokeratin
br1003a_1_cytokeratin_555_aSMA_647_hoechst_B7
aSMA
cytokeratin
br1003a_1_cytokeratin_555_aSMA_647_hoechst_B8
aSMA
cytokeratin
br1003a_1_cytokeratin_555_aSMA_647_hoechst_B9
aSMA
cytokeratin
br1003a_1_cytokeratin_555_aSMA_647_hoechst_D1
aSMA
cytokeratin
br1003a_1_cytokeratin_555_aSMA_647_hoechst_D2
aSMA
cytokeratin
br1003a_1_cytokeratin_555_aSMA_647_hoechst_D3
aSMA
cytokeratin
br1003a_1_cytokeratin_555_aSMA_647_hoechst_D4
aSMA
cytokeratin
br1003a_1_cytokeratin_555_aSMA_647_hoechst_D7
aSMA
cytokeratin
br1003a_1_cytokeratin_555_aSMA_647_hoechst_J1
aSMA
cytokeratin
br1003a_1_cytokeratin_555_aSMA_647_hoechst_J10
aSMA
cy

br8018a_4_cytokeratin_555_gh2ax_647_hoechst_C2
cytokeratin
gh2ax
br8018a_4_cytokeratin_555_gh2ax_647_hoechst_C4
cytokeratin
gh2ax
br8018a_4_cytokeratin_555_gh2ax_647_hoechst_C5
cytokeratin
gh2ax
br8018a_4_cytokeratin_555_gh2ax_647_hoechst_C6
cytokeratin
gh2ax
br8018a_4_cytokeratin_555_gh2ax_647_hoechst_C7
cytokeratin
gh2ax
br8018a_4_cytokeratin_555_gh2ax_647_hoechst_C8
cytokeratin
gh2ax
br8018a_4_cytokeratin_555_gh2ax_647_hoechst_C9
cytokeratin
gh2ax
br8018a_4_cytokeratin_555_gh2ax_647_hoechst_D1
cytokeratin
gh2ax
br8018a_4_cytokeratin_555_gh2ax_647_hoechst_D3
cytokeratin
gh2ax
br8018a_4_cytokeratin_555_gh2ax_647_hoechst_D5
cytokeratin
gh2ax
br8018a_4_cytokeratin_555_gh2ax_647_hoechst_D8
cytokeratin
gh2ax
br8018a_4_cytokeratin_555_gh2ax_647_hoechst_D9
cytokeratin
gh2ax
br8018a_4_cytokeratin_555_gh2ax_647_hoechst_E1
cytokeratin
gh2ax
br8018a_4_cytokeratin_555_gh2ax_647_hoechst_E3
cytokeratin
gh2ax
br8018a_4_cytokeratin_555_gh2ax_647_hoechst_E6
cytokeratin
gh2ax
br8018a_4_cytokeratin_555

In [19]:
#all
stats2plot=['int_median','kurtosis','skewness','int_mode','entropy']
stains=['aSMA','cytokeratin','collagen1','gh2ax','ki67']
uniquenames,nameIdx=np.unique(allImgNames_new,return_index=True)

allstats=None
alllabels=None
allvarnames=None
for sidx in range(uniquenames.size):
    s=np.unique(allImgNames_new)[sidx]
    plottingIdx_i_s=np.arange(allImgNames_new.size)[allImgNames_new==s]-nameIdx[sidx]

    print(s)
    assert np.min(plottingIdx_i_s)>=0
#         ssplit=s.split('_')
    slabels=clusterRes_reordered_new[allImgNames_new==s]
    for stain_s in stains:
        if not stain_s in s:
            continue
        path_s=os.path.join(datadir,'_'.join(s.split('_')[:-1]),'cellular_'+stain_s.lower(),s.split('_')[-1] +'.csv')
        if not os.path.exists(path_s):
            continue
        print(stain_s)
        stats_s=pd.read_csv(path_s)
        stats_s.index=stats_s.loc[:,'label']
        if allstats is None:
            allstats=stats_s.loc[cellIDlist_new[s][plottingIdx_i_s],stats2plot].to_numpy()
            alllabels=np.copy(slabels)
            allvarnames=np.repeat(stain_s,slabels.size)
        else:
            allstats=np.concatenate((allstats,stats_s.loc[cellIDlist_new[s][plottingIdx_i_s],stats2plot].to_numpy()),axis=0)
            alllabels=np.concatenate((alllabels,np.copy(slabels)))
            allvarnames=np.concatenate((allvarnames,np.repeat(stain_s,slabels.size)))
plotCTcomp_exprs(stats2plot,alllabels,allstats,allvarnames,clusterplotdir_valsamples,'proteinHeatmap_all',True)
plotCTcomp_exprs(stats2plot,alllabels,allstats,allvarnames,clusterplotdir_valsamples,'proteinHeatmap_all',False)

            

br1003a_2_cytokeratin_555_ki67_647_hoechst_A1
cytokeratin
ki67
br1003a_2_cytokeratin_555_ki67_647_hoechst_A10
cytokeratin
ki67
br1003a_2_cytokeratin_555_ki67_647_hoechst_A2
cytokeratin
ki67
br1003a_2_cytokeratin_555_ki67_647_hoechst_A3
cytokeratin
ki67
br1003a_2_cytokeratin_555_ki67_647_hoechst_A5
cytokeratin
ki67
br1003a_2_cytokeratin_555_ki67_647_hoechst_A7
cytokeratin
ki67
br1003a_2_cytokeratin_555_ki67_647_hoechst_A8
cytokeratin
ki67
br1003a_2_cytokeratin_555_ki67_647_hoechst_B1
cytokeratin
ki67
br1003a_2_cytokeratin_555_ki67_647_hoechst_B10
cytokeratin
ki67
br1003a_2_cytokeratin_555_ki67_647_hoechst_B2
cytokeratin
ki67
br1003a_2_cytokeratin_555_ki67_647_hoechst_B3
cytokeratin
ki67
br1003a_2_cytokeratin_555_ki67_647_hoechst_B4
cytokeratin
ki67
br1003a_2_cytokeratin_555_ki67_647_hoechst_B5
cytokeratin
ki67
br1003a_2_cytokeratin_555_ki67_647_hoechst_B6
cytokeratin
ki67
br1003a_2_cytokeratin_555_ki67_647_hoechst_B7
cytokeratin
ki67
br1003a_2_cytokeratin_555_ki67_647_hoechst_B8
cytoker

br8018a_2_cytokeratin_555_ki67_647_hoechst_D3
cytokeratin
ki67
br8018a_2_cytokeratin_555_ki67_647_hoechst_D4
cytokeratin
ki67
br8018a_2_cytokeratin_555_ki67_647_hoechst_D5
cytokeratin
ki67
br8018a_2_cytokeratin_555_ki67_647_hoechst_D6
cytokeratin
ki67
br8018a_2_cytokeratin_555_ki67_647_hoechst_D7
cytokeratin
ki67
br8018a_2_cytokeratin_555_ki67_647_hoechst_E1
cytokeratin
ki67
br8018a_2_cytokeratin_555_ki67_647_hoechst_E10
cytokeratin
ki67
br8018a_2_cytokeratin_555_ki67_647_hoechst_E2
cytokeratin
ki67
br8018a_2_cytokeratin_555_ki67_647_hoechst_E3
cytokeratin
ki67
br8018a_2_cytokeratin_555_ki67_647_hoechst_E4
cytokeratin
ki67
br8018a_2_cytokeratin_555_ki67_647_hoechst_E5
cytokeratin
ki67
br8018a_2_cytokeratin_555_ki67_647_hoechst_E6
cytokeratin
ki67
br8018a_2_cytokeratin_555_ki67_647_hoechst_E7
cytokeratin
ki67
br8018a_2_cytokeratin_555_ki67_647_hoechst_E8
cytokeratin
ki67
br8018a_2_cytokeratin_555_ki67_647_hoechst_E9
cytokeratin
ki67
br8018a_2_cytokeratin_555_ki67_647_hoechst_F1
cytokera

In [20]:
#equal proportion of pathologies
stats2plot=['int_median','kurtosis','skewness','int_mode','entropy']
stains=['aSMA','cytokeratin','collagen1','gh2ax','ki67']
uniquenames,nameIdx=np.unique(allImgNames_new,return_index=True)

allstats=None
alllabels=None
allvarnames=None
for sidx in range(uniquenames.size):
    s=np.unique(allImgNames_new)[sidx]
    plottingIdx_i_s=plottingIdx_prop_new[allImgNames_new[plottingIdx_prop_new]==s]-nameIdx[sidx]

    print(s)
    assert np.min(plottingIdx_i_s)>=0
#         ssplit=s.split('_')
    slabels=clusterRes_reordered_new[plottingIdx_prop_new][allImgNames_new[plottingIdx_prop_new]==s]
    for stain_s in stains:
        if not stain_s in s:
            continue
        path_s=os.path.join(datadir,'_'.join(s.split('_')[:-1]),'cellular_'+stain_s.lower(),s.split('_')[-1] +'.csv')
        if not os.path.exists(path_s):
            continue
        print(stain_s)
        stats_s=pd.read_csv(path_s)
        stats_s.index=stats_s.loc[:,'label']
        if allstats is None:
            allstats=stats_s.loc[cellIDlist_new[s][plottingIdx_i_s],stats2plot].to_numpy()
            alllabels=np.copy(slabels)
            allvarnames=np.repeat(stain_s,slabels.size)
        else:
            allstats=np.concatenate((allstats,stats_s.loc[cellIDlist_new[s][plottingIdx_i_s],stats2plot].to_numpy()),axis=0)
            alllabels=np.concatenate((alllabels,np.copy(slabels)))
            allvarnames=np.concatenate((allvarnames,np.repeat(stain_s,slabels.size)))
plotCTcomp_exprs(stats2plot,alllabels,allstats,allvarnames,clusterplotdir_valsamples,'proteinHeatmap_prop',True)
plotCTcomp_exprs(stats2plot,alllabels,allstats,allvarnames,clusterplotdir_valsamples,'proteinHeatmap_prop',False)

            

br1003a_2_cytokeratin_555_ki67_647_hoechst_A1
cytokeratin
ki67
br1003a_2_cytokeratin_555_ki67_647_hoechst_A10
cytokeratin
ki67
br1003a_2_cytokeratin_555_ki67_647_hoechst_A2
cytokeratin
ki67
br1003a_2_cytokeratin_555_ki67_647_hoechst_A3
cytokeratin
ki67
br1003a_2_cytokeratin_555_ki67_647_hoechst_A5
cytokeratin
ki67
br1003a_2_cytokeratin_555_ki67_647_hoechst_A7
cytokeratin
ki67
br1003a_2_cytokeratin_555_ki67_647_hoechst_A8
cytokeratin
ki67
br1003a_2_cytokeratin_555_ki67_647_hoechst_B1
cytokeratin
ki67
br1003a_2_cytokeratin_555_ki67_647_hoechst_B10
cytokeratin
ki67
br1003a_2_cytokeratin_555_ki67_647_hoechst_B2
cytokeratin
ki67
br1003a_2_cytokeratin_555_ki67_647_hoechst_B3
cytokeratin
ki67
br1003a_2_cytokeratin_555_ki67_647_hoechst_B4
cytokeratin
ki67
br1003a_2_cytokeratin_555_ki67_647_hoechst_B5
cytokeratin
ki67
br1003a_2_cytokeratin_555_ki67_647_hoechst_B6
cytokeratin
ki67
br1003a_2_cytokeratin_555_ki67_647_hoechst_B7
cytokeratin
ki67
br1003a_2_cytokeratin_555_ki67_647_hoechst_B8
cytoker

br8018a_2_cytokeratin_555_ki67_647_hoechst_D3
cytokeratin
ki67
br8018a_2_cytokeratin_555_ki67_647_hoechst_D4
cytokeratin
ki67
br8018a_2_cytokeratin_555_ki67_647_hoechst_D5
cytokeratin
ki67
br8018a_2_cytokeratin_555_ki67_647_hoechst_D6
cytokeratin
ki67
br8018a_2_cytokeratin_555_ki67_647_hoechst_D7
cytokeratin
ki67
br8018a_2_cytokeratin_555_ki67_647_hoechst_E1
cytokeratin
ki67
br8018a_2_cytokeratin_555_ki67_647_hoechst_E10
cytokeratin
ki67
br8018a_2_cytokeratin_555_ki67_647_hoechst_E2
cytokeratin
ki67
br8018a_2_cytokeratin_555_ki67_647_hoechst_E3
cytokeratin
ki67
br8018a_2_cytokeratin_555_ki67_647_hoechst_E4
cytokeratin
ki67
br8018a_2_cytokeratin_555_ki67_647_hoechst_E5
cytokeratin
ki67
br8018a_2_cytokeratin_555_ki67_647_hoechst_E6
cytokeratin
ki67
br8018a_2_cytokeratin_555_ki67_647_hoechst_E7
cytokeratin
ki67
br8018a_2_cytokeratin_555_ki67_647_hoechst_E8
cytokeratin
ki67
br8018a_2_cytokeratin_555_ki67_647_hoechst_E9
cytokeratin
ki67
br8018a_2_cytokeratin_555_ki67_647_hoechst_F1
cytokera

## duct membership histogram

In [23]:
uniquenames,nameIdx=np.unique(allImgNames,return_index=True)

allstats=None
alllabels=None
allprogs=None
for sidx in range(uniquenames.size):
    s=np.unique(allImgNames)[sidx]

    path_s=os.path.join(datadir,'_'.join(s.split('_')[:-1]),'duct_nuc_membership',s.split('_')[-1] +'.csv')
    if not os.path.exists(path_s):
        continue
    print(s)
    plottingIdx_i_s=np.arange(allImgNames.size)[allImgNames==s]-nameIdx[sidx]
    assert np.min(plottingIdx_i_s)>=0

    stats_s=pd.read_csv(path_s)
    stats_s.index=stats_s.loc[:,'label']
    stats_s=stats_s.loc[cellIDlist[s][plottingIdx_i_s],'int_median'].to_numpy()


#         ssplit=s.split('_')
    slabels=clusterRes_reordered[allImgNames==s]
    sprog=progList[allImgNames==s]

    if allstats is None:
        allstats=stats_s>0
        alllabels=np.copy(slabels)
        allprogs=np.copy(sprog)
    else:
        allstats=np.concatenate((allstats,stats_s>0))
        alllabels=np.concatenate((alllabels,np.copy(slabels)))
        allprogs=np.concatenate((allprogs,np.copy(sprog)))
for p in np.unique(allprogs):
    plotCTcomp_hist(alllabels[allprogs==p],allstats[allprogs==p],clusterplotdir_valcores,'ductHist'+p,False)
            

br1003a_1_cytokeratin_555_aSMA_647_hoechst_B1
br1003a_1_cytokeratin_555_aSMA_647_hoechst_B2
br1003a_1_cytokeratin_555_aSMA_647_hoechst_B3
br1003a_1_cytokeratin_555_aSMA_647_hoechst_B5
br1003a_1_cytokeratin_555_aSMA_647_hoechst_B7
br8018a_1_cytokeratin_555_aSMA_647_hoechst_D10
br8018a_1_cytokeratin_555_aSMA_647_hoechst_D6
br8018a_1_cytokeratin_555_aSMA_647_hoechst_D7
br8018a_1_cytokeratin_555_aSMA_647_hoechst_E5
br8018a_1_cytokeratin_555_aSMA_647_hoechst_E6
br8018a_1_cytokeratin_555_aSMA_647_hoechst_E7
br8018a_1_cytokeratin_555_aSMA_647_hoechst_E8
br8018a_1_cytokeratin_555_aSMA_647_hoechst_E9


In [24]:
uniquenames,nameIdx=np.unique(allImgNames_new,return_index=True)

allstats=None
alllabels=None
allprogs=None
for sidx in range(uniquenames.size):
    s=np.unique(allImgNames_new)[sidx]

    path_s=os.path.join(datadir,'_'.join(s.split('_')[:-1]),'duct_nuc_membership',s.split('_')[-1] +'.csv')
    if not os.path.exists(path_s):
        continue
    print(s)
    plottingIdx_i_s=np.arange(allImgNames_new.size)[allImgNames_new==s]-nameIdx[sidx]
    assert np.min(plottingIdx_i_s)>=0

    stats_s=pd.read_csv(path_s)
    stats_s.index=stats_s.loc[:,'label']
    stats_s=stats_s.loc[cellIDlist_new[s][plottingIdx_i_s],'int_median'].to_numpy()


#         ssplit=s.split('_')
    slabels=clusterRes_reordered_new[allImgNames_new==s]
    sprog=progList_new[allImgNames_new==s]

    if allstats is None:
        allstats=stats_s>0
        alllabels=np.copy(slabels)
        allprogs=np.copy(sprog)
    else:
        allstats=np.concatenate((allstats,stats_s>0))
        alllabels=np.concatenate((alllabels,np.copy(slabels)))
        allprogs=np.concatenate((allprogs,np.copy(sprog)))
for p in np.unique(allprogs):
    plotCTcomp_hist(alllabels[allprogs==p],allstats[allprogs==p],clusterplotdir_valsamples,'ductHist'+p,False)
            

br1003a_2_cytokeratin_555_ki67_647_hoechst_B1
br1003a_2_cytokeratin_555_ki67_647_hoechst_B2
br1003a_2_cytokeratin_555_ki67_647_hoechst_B3
br1003a_2_cytokeratin_555_ki67_647_hoechst_B5
br1003a_2_cytokeratin_555_ki67_647_hoechst_B7
br1003a_2_cytokeratin_555_ki67_647_hoechst_I2
br1003a_2_cytokeratin_555_ki67_647_hoechst_I3
br1003a_2_cytokeratin_555_ki67_647_hoechst_I7
br1003a_2_cytokeratin_555_ki67_647_hoechst_I8
br1003a_2_cytokeratin_555_ki67_647_hoechst_I9
br301_5_cytokeratin_555_ki67_647_hoechst_A1
br301_5_cytokeratin_555_ki67_647_hoechst_A2
br301_5_cytokeratin_555_ki67_647_hoechst_A3
br301_5_cytokeratin_555_ki67_647_hoechst_A4
br301_5_cytokeratin_555_ki67_647_hoechst_A5
br301_5_cytokeratin_555_ki67_647_hoechst_A6
br301_5_cytokeratin_555_ki67_647_hoechst_B1
br301_5_cytokeratin_555_ki67_647_hoechst_B2
br301_5_cytokeratin_555_ki67_647_hoechst_B3
br301_5_cytokeratin_555_ki67_647_hoechst_B4
br301_5_cytokeratin_555_ki67_647_hoechst_B5
br301_5_cytokeratin_555_ki67_647_hoechst_B6
br301_5_cyto

#### subclusters

In [25]:
uniquenames,nameIdx=np.unique(allImgNames,return_index=True)

for cnew in np.unique(clusterRes_reordered):
    c=neworder[cnew]
    subclustersavedir_valcores=os.path.join(clustersavedir_valcores,savenamecluster+plotsavenameAdd_i+'_subcluster'+str(c))
    subclusterplotdir_valcores=os.path.join(subclustersavedir_valcores,'plots')
    with open(os.path.join(subclustersavedir_valcores,'minibatchkmean_ncluster'+str(subclusternumbers[c])+'n_pcs'+str(n_pcs)+'epoch'+str(plotepoch)+'_all'), 'rb') as output:
        subclusterRes=pickle.load(output)
    imgNames_i=allImgNames[clusterRes_reordered==cnew]
    plottingIdx_i_i=np.arange(clusterRes_reordered.size)[clusterRes_reordered==cnew]

    allstats=None
    alllabels=None
    allprogs=None
    for sidx in range(uniquenames.size):
        s=np.unique(allImgNames)[sidx]

        path_s=os.path.join(datadir,'_'.join(s.split('_')[:-1]),'duct_nuc_membership',s.split('_')[-1] +'.csv')
        if not os.path.exists(path_s):
            continue
        print(s)
        plottingIdx_i_s=plottingIdx_i_i[imgNames_i==s]-nameIdx[sidx]
        if plottingIdx_i_s.size==0:
            continue
        assert np.min(plottingIdx_i_s)>=0

        stats_s=pd.read_csv(path_s)
        stats_s.index=stats_s.loc[:,'label']
        stats_s=stats_s.loc[cellIDlist[s][plottingIdx_i_s],'int_median'].to_numpy()


#         ssplit=s.split('_')
        slabels=subclusterRes[imgNames_i==s]
        sprog=progList[plottingIdx_i_i][imgNames_i==s]

        if allstats is None:
            allstats=stats_s>0
            alllabels=np.copy(slabels)
            allprogs=np.copy(sprog)
        else:
            allstats=np.concatenate((allstats,stats_s>0))
            alllabels=np.concatenate((alllabels,np.copy(slabels)))
            allprogs=np.concatenate((allprogs,np.copy(sprog)))
    for p in np.unique(allprogs):
        plotCTcomp_hist(alllabels[allprogs==p],allstats[allprogs==p],subclusterplotdir_valcores,'ductHist'+p+'_ncluster'+str(subclusternumbers[c]),False)

br1003a_1_cytokeratin_555_aSMA_647_hoechst_B1
br1003a_1_cytokeratin_555_aSMA_647_hoechst_B2
br1003a_1_cytokeratin_555_aSMA_647_hoechst_B3
br1003a_1_cytokeratin_555_aSMA_647_hoechst_B5
br1003a_1_cytokeratin_555_aSMA_647_hoechst_B7
br8018a_1_cytokeratin_555_aSMA_647_hoechst_D10
br8018a_1_cytokeratin_555_aSMA_647_hoechst_D6
br8018a_1_cytokeratin_555_aSMA_647_hoechst_D7
br8018a_1_cytokeratin_555_aSMA_647_hoechst_E5
br8018a_1_cytokeratin_555_aSMA_647_hoechst_E6
br8018a_1_cytokeratin_555_aSMA_647_hoechst_E7
br8018a_1_cytokeratin_555_aSMA_647_hoechst_E8
br8018a_1_cytokeratin_555_aSMA_647_hoechst_E9
br1003a_1_cytokeratin_555_aSMA_647_hoechst_B1
br1003a_1_cytokeratin_555_aSMA_647_hoechst_B2
br1003a_1_cytokeratin_555_aSMA_647_hoechst_B3
br1003a_1_cytokeratin_555_aSMA_647_hoechst_B5
br1003a_1_cytokeratin_555_aSMA_647_hoechst_B7
br8018a_1_cytokeratin_555_aSMA_647_hoechst_D10
br8018a_1_cytokeratin_555_aSMA_647_hoechst_D6
br8018a_1_cytokeratin_555_aSMA_647_hoechst_D7
br8018a_1_cytokeratin_555_aSMA_6

In [26]:
uniquenames,nameIdx=np.unique(allImgNames_new,return_index=True)

for cnew in np.unique(clusterRes_reordered_new):
    c=neworder[cnew]
    subclustersavedir_valsamples=os.path.join(clustersavedir_valsamples,savenamecluster+plotsavenameAdd_i+'_subcluster'+str(c))
    subclusterplotdir_valsamples=os.path.join(subclustersavedir_valsamples,'plots')
    with open(os.path.join(subclustersavedir_valsamples,'minibatchkmean_ncluster'+str(subclusternumbers[c])+'n_pcs'+str(n_pcs)+'epoch'+str(plotepoch)+'_all'), 'rb') as output:
        subclusterRes=pickle.load(output)
    imgNames_i=allImgNames_new[clusterRes_reordered_new==cnew]
    plottingIdx_i_i=np.arange(clusterRes_reordered_new.size)[clusterRes_reordered_new==cnew]

    allstats=None
    alllabels=None
    allprogs=None
    for sidx in range(uniquenames.size):
        s=np.unique(allImgNames_new)[sidx]

        path_s=os.path.join(datadir,'_'.join(s.split('_')[:-1]),'duct_nuc_membership',s.split('_')[-1] +'.csv')
        if not os.path.exists(path_s):
            continue
        print(s)
        plottingIdx_i_s=plottingIdx_i_i[imgNames_i==s]-nameIdx[sidx]
        if plottingIdx_i_s.size==0:
            continue
        assert np.min(plottingIdx_i_s)>=0

        stats_s=pd.read_csv(path_s)
        stats_s.index=stats_s.loc[:,'label']
        stats_s=stats_s.loc[cellIDlist_new[s][plottingIdx_i_s],'int_median'].to_numpy()


#         ssplit=s.split('_')
        slabels=subclusterRes[imgNames_i==s]
        sprog=progList_new[plottingIdx_i_i][imgNames_i==s]

        if allstats is None:
            allstats=stats_s>0
            alllabels=np.copy(slabels)
            allprogs=np.copy(sprog)
        else:
            allstats=np.concatenate((allstats,stats_s>0))
            alllabels=np.concatenate((alllabels,np.copy(slabels)))
            allprogs=np.concatenate((allprogs,np.copy(sprog)))
    for p in np.unique(allprogs):
        plotCTcomp_hist(alllabels[allprogs==p],allstats[allprogs==p],subclusterplotdir_valsamples,'ductHist'+p+'_ncluster'+str(subclusternumbers[c]),False)

br1003a_2_cytokeratin_555_ki67_647_hoechst_B1
br1003a_2_cytokeratin_555_ki67_647_hoechst_B2
br1003a_2_cytokeratin_555_ki67_647_hoechst_B3
br1003a_2_cytokeratin_555_ki67_647_hoechst_B5
br1003a_2_cytokeratin_555_ki67_647_hoechst_B7
br1003a_2_cytokeratin_555_ki67_647_hoechst_I2
br1003a_2_cytokeratin_555_ki67_647_hoechst_I3
br1003a_2_cytokeratin_555_ki67_647_hoechst_I7
br1003a_2_cytokeratin_555_ki67_647_hoechst_I8
br1003a_2_cytokeratin_555_ki67_647_hoechst_I9
br301_5_cytokeratin_555_ki67_647_hoechst_A1
br301_5_cytokeratin_555_ki67_647_hoechst_A2
br301_5_cytokeratin_555_ki67_647_hoechst_A3
br301_5_cytokeratin_555_ki67_647_hoechst_A4
br301_5_cytokeratin_555_ki67_647_hoechst_A5
br301_5_cytokeratin_555_ki67_647_hoechst_A6
br301_5_cytokeratin_555_ki67_647_hoechst_B1
br301_5_cytokeratin_555_ki67_647_hoechst_B2
br301_5_cytokeratin_555_ki67_647_hoechst_B3
br301_5_cytokeratin_555_ki67_647_hoechst_B4
br301_5_cytokeratin_555_ki67_647_hoechst_B5
br301_5_cytokeratin_555_ki67_647_hoechst_B6
br301_5_cyto

br301_5_cytokeratin_555_ki67_647_hoechst_D6
br301_5_cytokeratin_555_ki67_647_hoechst_E1
br301_5_cytokeratin_555_ki67_647_hoechst_E2
br301_5_cytokeratin_555_ki67_647_hoechst_E3
br301_5_cytokeratin_555_ki67_647_hoechst_E4
br301_5_cytokeratin_555_ki67_647_hoechst_E5
br301_5_cytokeratin_555_ki67_647_hoechst_E6
br8018a_2_cytokeratin_555_ki67_647_hoechst_D10
br8018a_2_cytokeratin_555_ki67_647_hoechst_D6
br8018a_2_cytokeratin_555_ki67_647_hoechst_D7
br8018a_2_cytokeratin_555_ki67_647_hoechst_E5
br8018a_2_cytokeratin_555_ki67_647_hoechst_E6
br8018a_2_cytokeratin_555_ki67_647_hoechst_E7
br8018a_2_cytokeratin_555_ki67_647_hoechst_E8
br8018a_2_cytokeratin_555_ki67_647_hoechst_E9
br8018a_2_cytokeratin_555_ki67_647_hoechst_F2
br8018a_2_cytokeratin_555_ki67_647_hoechst_F7
br1003a_2_cytokeratin_555_ki67_647_hoechst_B1
br1003a_2_cytokeratin_555_ki67_647_hoechst_B2
br1003a_2_cytokeratin_555_ki67_647_hoechst_B3
br1003a_2_cytokeratin_555_ki67_647_hoechst_B5
br1003a_2_cytokeratin_555_ki67_647_hoechst_B7
b

br301_5_cytokeratin_555_ki67_647_hoechst_B1
br301_5_cytokeratin_555_ki67_647_hoechst_B2
br301_5_cytokeratin_555_ki67_647_hoechst_B3
br301_5_cytokeratin_555_ki67_647_hoechst_B4
br301_5_cytokeratin_555_ki67_647_hoechst_B5
br301_5_cytokeratin_555_ki67_647_hoechst_B6
br301_5_cytokeratin_555_ki67_647_hoechst_C1
br301_5_cytokeratin_555_ki67_647_hoechst_C2
br301_5_cytokeratin_555_ki67_647_hoechst_C3
br301_5_cytokeratin_555_ki67_647_hoechst_C4
br301_5_cytokeratin_555_ki67_647_hoechst_C5
br301_5_cytokeratin_555_ki67_647_hoechst_C6
br301_5_cytokeratin_555_ki67_647_hoechst_D1
br301_5_cytokeratin_555_ki67_647_hoechst_D2
br301_5_cytokeratin_555_ki67_647_hoechst_D3
br301_5_cytokeratin_555_ki67_647_hoechst_D4
br301_5_cytokeratin_555_ki67_647_hoechst_D5
br301_5_cytokeratin_555_ki67_647_hoechst_D6
br301_5_cytokeratin_555_ki67_647_hoechst_E1
br301_5_cytokeratin_555_ki67_647_hoechst_E2
br301_5_cytokeratin_555_ki67_647_hoechst_E3
br301_5_cytokeratin_555_ki67_647_hoechst_E4
br301_5_cytokeratin_555_ki67_647

### plot nuclei in each clusters

In [27]:
with open(os.path.join(datadir,'processed','train_cnnvae_valcores'), 'rb') as input:
    allImg=pickle.load(input)

In [28]:
#Sample 10 nuclei/pathology from each cluster
ncells=20

progs=np.unique(progList)
for p in range(progs.size):
    img_p=allImg[progList==progs[p]]
    cluster_p=clusterRes_reordered[progList==progs[p]]
    plotExample=np.zeros((np.unique(cluster_p).size*(radius+1)*2,ncells*(radius+1)*2))
    for c in range(np.unique(cluster_p).size):
        clusteridx=np.arange(img_p.shape[0])[cluster_p==np.unique(cluster_p)[c]]
        np.random.seed(6)
        cp_plot=np.random.choice(clusteridx,min(ncells,clusteridx.size),replace=False)
        for cp_i in range(cp_plot.size):
            plotExample[c*(radius*2+2):c*(radius*2+2)+radius*2,cp_i*(radius*2+2):cp_i*(radius*2+2)+radius*2]=img_p[cp_plot[cp_i]]
    fig, ax = plt.subplots(figsize=(20,20))
    img = ax.imshow(plotExample)
    ax.set_yticks(np.arange(np.unique(cluster_p).size).astype(int)*(radius*2+2)+(radius+1))
    ax.set_yticklabels(np.unique(cluster_p))
    fig.tight_layout()
    plt.savefig(os.path.join(clusterplotdir_valcores,'plotexamples'+progs[p]+'.jpg'))
    plt.close()

#plot examples
for cnew in np.unique(clusterRes_reordered):
    c=neworder[cnew]
    subclustersavedir_valcores=os.path.join(clustersavedir_valcores,savenamecluster+plotsavenameAdd_i+'_subcluster'+str(c))
    subclusterplotdir_valcores=os.path.join(subclustersavedir_valcores,'plots')
    with open(os.path.join(subclustersavedir_valcores,'minibatchkmean_ncluster'+str(subclusternumbers[c])+'n_pcs'+str(n_pcs)+'epoch'+str(plotepoch)+'_all'), 'rb') as output:
        subclusterRes=pickle.load(output)

    progs=np.unique(progList[clusterRes_reordered==cnew])
    for p in range(progs.size):
        img_p=allImg[clusterRes_reordered==cnew][progList[clusterRes_reordered==cnew]==progs[p]]
        cluster_p=subclusterRes[progList[clusterRes_reordered==cnew]==progs[p]]
        plotExample=np.zeros((np.unique(cluster_p).size*(radius+1)*2,ncells*(radius+1)*2))
        for csub in range(np.unique(cluster_p).size):
            clusteridx=np.arange(img_p.shape[0])[cluster_p==np.unique(cluster_p)[csub]]
            np.random.seed(6)
            cp_plot=np.random.choice(clusteridx,min(ncells,clusteridx.size),replace=False)
            for cp_i in range(cp_plot.size):
                plotExample[csub*(radius*2+2):csub*(radius*2+2)+radius*2,cp_i*(radius*2+2):cp_i*(radius*2+2)+radius*2]=img_p[cp_plot[cp_i]]
        fig, ax = plt.subplots(figsize=(20,20))
        img = ax.imshow(plotExample)
        ax.set_yticks(np.arange(np.unique(cluster_p).size).astype(int)*(radius*2+2)+(radius+1))
        ax.set_yticklabels(np.unique(cluster_p))
        fig.tight_layout()
        plt.savefig(os.path.join(subclusterplotdir_valcores,'plotexamples'+progs[p]+str(subclusternumbers[c])+'.jpg'))
        plt.close()
        gc.collect()
    
allImg=None
gc.collect()

0

In [29]:
with open(os.path.join(datadir,'processed','train_cnnvae_valsamples'), 'rb') as input:
    allImg=pickle.load(input)

In [30]:
#Sample 10 nuclei/pathology from each cluster
ncells=20

progs=np.unique(progList_new)
for p in range(progs.size):
    img_p=allImg[progList_new==progs[p]]
    cluster_p=clusterRes_reordered_new[progList_new==progs[p]]
    plotExample=np.zeros((np.unique(cluster_p).size*(radius+1)*2,ncells*(radius+1)*2))
    for c in range(np.unique(cluster_p).size):
        clusteridx=np.arange(img_p.shape[0])[cluster_p==np.unique(cluster_p)[c]]
        np.random.seed(6)
        cp_plot=np.random.choice(clusteridx,min(ncells,clusteridx.size),replace=False)
        for cp_i in range(cp_plot.size):
            plotExample[c*(radius*2+2):c*(radius*2+2)+radius*2,cp_i*(radius*2+2):cp_i*(radius*2+2)+radius*2]=img_p[cp_plot[cp_i]]
    fig, ax = plt.subplots(figsize=(20,20))
    img = ax.imshow(plotExample)
    ax.set_yticks(np.arange(np.unique(cluster_p).size).astype(int)*(radius*2+2)+(radius+1))
    ax.set_yticklabels(np.unique(cluster_p))
    fig.tight_layout()
    plt.savefig(os.path.join(clusterplotdir_valsamples,'plotexamples'+progs[p]+'.jpg'))
    plt.close()

#plot examples
for cnew in np.unique(clusterRes_reordered_new):
    c=neworder[cnew]
    subclustersavedir_valsamples=os.path.join(clustersavedir_valsamples,savenamecluster+plotsavenameAdd_i+'_subcluster'+str(c))
    subclusterplotdir_valsamples=os.path.join(subclustersavedir_valsamples,'plots')
    with open(os.path.join(subclustersavedir_valsamples,'minibatchkmean_ncluster'+str(subclusternumbers[c])+'n_pcs'+str(n_pcs)+'epoch'+str(plotepoch)+'_all'), 'rb') as output:
        subclusterRes=pickle.load(output)

    progs=np.unique(progList_new[clusterRes_reordered_new==cnew])
    for p in range(progs.size):
        img_p=allImg[clusterRes_reordered_new==cnew][progList_new[clusterRes_reordered_new==cnew]==progs[p]]
        cluster_p=subclusterRes[progList_new[clusterRes_reordered_new==cnew]==progs[p]]
        plotExample=np.zeros((np.unique(cluster_p).size*(radius+1)*2,ncells*(radius+1)*2))
        for csub in range(np.unique(cluster_p).size):
            clusteridx=np.arange(img_p.shape[0])[cluster_p==np.unique(cluster_p)[csub]]
            np.random.seed(6)
            cp_plot=np.random.choice(clusteridx,min(ncells,clusteridx.size),replace=False)
            for cp_i in range(cp_plot.size):
                plotExample[csub*(radius*2+2):csub*(radius*2+2)+radius*2,cp_i*(radius*2+2):cp_i*(radius*2+2)+radius*2]=img_p[cp_plot[cp_i]]
        fig, ax = plt.subplots(figsize=(20,20))
        img = ax.imshow(plotExample)
        ax.set_yticks(np.arange(np.unique(cluster_p).size).astype(int)*(radius*2+2)+(radius+1))
        ax.set_yticklabels(np.unique(cluster_p))
        fig.tight_layout()
        plt.savefig(os.path.join(subclusterplotdir_valsamples,'plotexamples'+progs[p]+str(subclusternumbers[c])+'.jpg'))
        plt.close()
        gc.collect()
    
allImg=None
gc.collect()

0